## Intro

In [20]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
os.chdir('/Users/timothymiller/Desktop/Dataset-SS')

In [3]:
# excel file of 4 sheets
file = pd.ExcelFile('schoolshootingdatabase.xlsx')

In [4]:
all_sheets = file.sheet_names # gets list of all sheet names
dfs = {sheet: file.parse(sheet) for sheet in all_sheets} # dictionary of dataframes for each sheet

In [5]:
for sheetname in all_sheets:
    print(sheetname)

Cover
Incident
Shooter
Victim
Weapon


In [6]:
incident = dfs['Incident']
shooter = dfs['Shooter']
victim = dfs['Victim']
weapon = dfs['Weapon']

# Incident Dataset

## EDA / Cleanup

In [7]:
incident.head()

,Incident_ID,Month,Day,Year,Date,School,Victims_Killed,Victims_Wounded,Number_Victims,Shooter_Killed,...,Situation,Targets,Accomplice,Hostages,Barricade,Officer_Involved,Bullied,Domestic_Violence,Gang_Related,Active_Shooter_FBI
0,20230329GANED,3,29,2023,2023-03-29,New Manchester High School,0,0,0,1,...,NaN,Neither,No,No,No,No,No,No,No,No
1,20230327TNCON,3,27,2023,2023-03-27,Covenant School,6,1,7,1,...,Indiscriminate Shooting,Random Shooting,No,No,No,No,NaN,No,No,Yes
2,20230324INNOM,3,24,2023,2023-03-24,Northridge Middle School,0,0,0,1,...,Suicide/Attempted,Victims Targeted,No,No,No,No,No,No,No,No
3,20230324NJTHM,3,24,2023,2023-03-24,Thunderbolt Academy,0,0,0,0,...,NaN,Victims Targeted,No,No,No,No,No,No,NaN,No
4,20230322ALANA,3,22,2023,2023-03-22,Anniston High School,0,1,1,0,...,Accidental,Neither,No,No,No,No,No,No,No,No


In [8]:
columns = incident.columns
print(columns)

Index(['Incident_ID', 'Month', 'Day', 'Year', 'Date', 'School',
       'Victims_Killed', 'Victims_Wounded', 'Number_Victims', 'Shooter_Killed',
       'Source', 'Number_News', 'Media_Attention', 'Reliability', 'Quarter',
       'City', 'State', 'School_Level', 'Location', 'Location_Type',
       'During_School', 'Time_Period', 'First_Shot', 'Duration_min', 'Summary',
       'Narrative', 'Situation', 'Targets', 'Accomplice', 'Hostages',
       'Barricade', 'Officer_Involved', 'Bullied', 'Domestic_Violence',
       'Gang_Related', 'Active_Shooter_FBI'],
      dtype='object')


In [25]:
count_by_year = incident['Year'].value_counts().reset_index()

In [26]:
count_by_situation = incident['Situation'].value_counts().reset_index()

In [31]:
count_by_targets = incident['Targets'].value_counts().reset_index()

Victims Targeted    1143
Random Shooting      318
Both                 249
Neither              238
Name: Targets, dtype: int64

In [10]:
# specify dataframe of incidents where there is at least 1 fatality
fatal_incidents = incident[(incident['Victims_Killed'] >= 1) | (incident['Shooter_Killed'] >= 1)]

In [11]:
incident.dtypes

Incident_ID                   object
Month                          int64
Day                            int64
Year                           int64
Date                  datetime64[ns]
School                        object
Victims_Killed                 int64
Victims_Wounded                int64
Number_Victims                 int64
Shooter_Killed                 int64
Source                        object
Number_News                  float64
Media_Attention               object
Reliability                    int64
Quarter                       object
City                          object
State                         object
School_Level                  object
Location                      object
Location_Type                 object
During_School                 object
Time_Period                   object
First_Shot                    object
Duration_min                 float64
Summary                       object
Narrative                     object
Situation                     object
T

In [12]:
# number of shootings in each state
state_counts = incident['State'].value_counts()

CA    229
TX    191
FL    131
IL    121
OH    104
PA    102
MI     96
NY     95
GA     83
NC     78
TN     75
MD     71
LA     66
AL     64
VA     56
MO     53
WA     49
IN     47
SC     45
DC     39
CO     36
WI     33
AR     32
OR     29
MS     28
KY     27
CT     24
NJ     23
NM     22
MA     22
MN     22
AZ     22
OK     22
UT     20
KS     20
IA     19
NV     19
DE     15
NE     11
MT     10
NH      8
ID      8
RI      7
AK      6
WV      6
ME      5
SD      5
VT      4
ND      3
HI      3
WY      2
VI      1
Name: State, dtype: int64


In [28]:
state_counts_fatal = fatal_incidents['State'].value_counts()

In [14]:
# Convert the series to dataframes
state_counts_df = state_counts.reset_index()
state_counts_fatal_df = state_counts_fatal.reset_index()

# Rename the columns
state_counts_df.columns = ['State', 'Total School Shootings']
state_counts_fatal_df.columns = ['State', 'Fatal School Shootings']

# Merge the dataframes
result = pd.merge(state_counts_df, state_counts_fatal_df, on='State')

print(result)

   State  Total School Shootings  Fatal School Shootings
0     CA                     229                      92
1     TX                     191                      71
2     FL                     131                      41
3     IL                     121                      38
4     OH                     104                      24
5     PA                     102                      37
6     MI                      96                      23
7     NY                      95                      23
8     GA                      83                      21
9     NC                      78                      16
10    TN                      75                      25
11    MD                      71                      19
12    LA                      66                      19
13    AL                      64                      12
14    VA                      56                      13
15    MO                      53                      21
16    WA                      4

In [15]:
result['Fatal / Total'] = result['Fatal School Shootings']/result['Total School Shootings']

In [30]:
result.sort_values(by='Fatal / Total', ascending=False)

,State,Total School Shootings,Fatal School Shootings,Fatal / Total
48,WY,2,2,1.000000
45,ME,5,3,0.600000
46,VT,4,2,0.500000
19,DC,39,18,0.461538
16,WA,49,22,0.448980
42,RI,7,3,0.428571
35,IA,19,8,0.421053
20,CO,36,15,0.416667
31,AZ,22,9,0.409091
0,CA,229,92,0.401747


# Dashboard

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': 'Montreal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
